In [1]:
import Pkg; 
cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`
Precompiling MorphoMol
  ✓ Distributions → DistributionsTestExt
  ✓ CairoMakie
  ✓ MorphoMol
  3 dependencies successfully precompiled in 18 seconds. 241 already precompiled.
[ Info: Precompiling MorphoMol [85e20460-a9b2-48f6-9df6-e154e9748d83]


In [2]:
@load "../../Data/jld2/2_experimental/2_6r7m_protor.jld2"

3-element Vector{Symbol}:
 :template_mol
 :template_radii
 :x_init

In [3]:
experimental_templates = Dict()
folder = "../../Data/jld2/2_experimental/"
for file in readdir(folder)
    mol_type = split(file, "_")[2]
    @load "$(folder)$(file)" template_mol template_radii x_init
    experimental_templates[mol_type] = Dict("template_centers" => template_mol, "template_radii" => template_radii, "state" => x_init)
end
open("2su_experimental_assembly.jl", "w") do file
    write(file, "TWOTMVSU_EXPERIMENTAL_ASSEMBLY = $(experimental_templates)")
end

217257

In [4]:
@load "../../Data/jld2/single_subunits/2tmv_protor.jld2"

3-element Vector{Symbol}:
 :template_mol
 :template_radii
 :x_init

In [5]:
experimental_templates = Dict()
folder = "../../Data/jld2/single_subunits/"
for file in readdir(folder)
    mol_type = split(file, "_")[1]
    @load "$(folder)$(file)" template_mol template_radii x_init
    experimental_templates[mol_type] = Dict("template_centers" => template_mol, "template_radii" => template_radii)
end
open("tmv_templates.jl", "w") do file
    write(file, "TMV_TEMPLATES = $(experimental_templates)")
end

447705